In [17]:
import pandas as pd
import statsapi
import datetime
import string
import json
import os

from bs4 import BeautifulSoup
import requests


In [24]:
player_search = pd.read_csv("data\\player_search.csv", encoding = "ISO-8859-1")
team_search = pd.read_csv("data\\team_search.csv")

wOBA_constants = pd.read_csv("data\\wOBA_constants.csv", index_col=0)
park_factors = pd.read_csv("data\\park_factors.csv", index_col=0)
league_averages = pd.read_csv("data\\league_averages.csv", index_col=0)

with open('data\\batter_hands.json') as f:
    batter_hands = json.load(f)
with open('data\\pitcher_hands.json') as f:
    pitcher_hands = json.load(f)


In [ ]:
def adv_matchup(row):
    val = 0
    if row['batter_hand'] == 'S':
        val = 1
    elif row['batter_hand'] == 'R' and row['pitcher_hand'] == 'L':
        val = 1
    elif row['batter_hand'] == 'L' and row['pitcher_hand'] == 'R':
        val = 1
    return val

for file in os.scandir("data\\matchups"):
    matchups = pd.read_csv(file, index_col=0)
    matchups['adv_ab'] = matchups.apply(adv_matchup, axis=1)
    matchups.to_csv(f"data\\matchups\\{file.name}")


In [18]:
matchups = pd.read_csv("data\\matchups\\2015_matchups.csv")

In [26]:
# Platoon statistics
for index, row in matchups.iterrows():
    platoon_history = matchups.loc[(matchups['batter_id'] == row['batter_id']) & (matchups['pitcher_hand'] == row['pitcher_hand'])]

    num_pa = len(platoon_history)
    outcomes = platoon_history.groupby(['outcome']).size()
    indexes = outcomes.index.tolist()

    singles = 0
    for event in ['h_S1','h_S2','h_S3','h_S4','h_S5','h_S6','h_S7','h_S8','h_S9']:
        if event in indexes:
            singles += outcomes[event]

    doubles = 0
    for event in ['h_D1','h_D2','h_D3','h_D4','h_D5','h_D6','h_D7','h_D8','h_D9']:
        if event in indexes:
            singles += outcomes[event]

    triples = 0
    for event in ['h_T1','h_T2','h_T3','h_T4','h_T5','h_T6','h_T7','h_T8','h_T9']:
        if event in indexes:
            singles += outcomes[event]

    home_runs = 0
    if 'h_HR' in indexes:
        home_runs += outcomes['h_HR']

    bb = 0
    if 'p_W' in indexes:
        bb += outcomes['p_W']

    ibb = 0
    if 'IW' in indexes:
        ibb += outcomes['IW']

    hbp = 0
    if 'HP' in indexes:
        hbp += outcomes['HP']

    at_bats = outcomes.sum()-bb-hbp

    wOBA_row = wOBA_constants.loc['Average']
    wOBA_num = (wOBA_row['wBB']*bb)+(wOBA_row['wHBP']*hbp)+(wOBA_row['w1B']*singles)+(wOBA_row['w2B']*doubles)+(wOBA_row['w3B']*triples)+(wOBA_row['wHR']*home_runs)
    wOBA_denom = at_bats + (bb - ibb) + hbp

    if wOBA_denom:
        wOBA = round(wOBA_num/wOBA_denom, 3)
    else:
        wOBA = 0

    print()


KeyboardInterrupt: 